In [ ]:
# change to the root directory of the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')
print(os.getcwd())

If we want to run this notebook on Google Colab, we first have to install `ScanDy` and download (an example version of) the dataset and the results from the evolutionary parameter optimization for all models from Google drive. The following code cell will prepare all of this for us.

In [ ]:
# install the ScanDy framework via pip
!pip install scandy

# download the VidCom_example dataset from google drive using gdown
!pip install gdown
# dataset is stored at https://drive.google.com/file/d/14kJDD_5ECP2vhgonhn5iQ5qnREDIqTTr/view?usp=sharing 
file_id = '14kJDD_5ECP2vhgonhn5iQ5qnREDIqTTr'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'vidcom_example.zip'
!gdown $url -O $output
!unzip $output

# Manuscript evolution results are stored at https://drive.google.com/file/d/1nVPwJEdJOAE-lbQhRteIai1g5YKmIgdJ/view?usp=sharing 
file_id = '1nVPwJEdJOAE-lbQhRteIai1g5YKmIgdJ'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'manuscript_results.zip'
!gdown $url -O $output
!unzip $output

In [ ]:
file_id = '1nVPwJEdJOAE-lbQhRteIai1g5YKmIgdJ'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'manuscript_results.zip'
!gdown $url -O $output
!unzip $output

In [ ]:
import numpy as np
import pandas as pd
import random
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.colors import to_rgb
from matplotlib.legend_handler import HandlerTuple

from scandy.models.LocationModel import LocationModel
from scandy.models.ObjectModel import ObjectModel
from scandy.utils.dataclass import Dataset
import scandy.utils.functions as uf

from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.evolution import Evolution

Load the dataset with the randomly generated train-test split used in the paper.

In [ ]:
vidlist = ['field03', 'dance01', 'dance02', 'foutain02', 'garden04', 'garden06', 'garden07', 'garden09', 'park01', 'park06', 'park09', 'road02', 'road04', 'road05', 'robarm01', 'room01', 'room02', 'room03', 'tommy02', 'uscdog01', 'walkway01', 'walkway02', 'walkway03']
random.seed(12345)
trainlist = sorted(random.sample(sorted(vidlist), 10))
testlist = sorted([vidname for vidname in vidlist if vidname not in trainlist])
print("trainlist = ", trainlist, "\ntestlist =", testlist)

datadict = {
    "PATH": "VidCom_example/",  # previously downloaded & extracted dataset  
    'FPS' : 30,
    'PX_TO_DVA' : 0.06,
    'FRAMES_ALL_VIDS' : 300,
    'gt_foveation_df' : '2021-12-04_VidCom_GT_fov_df',
    # hacky way of initializing, necessary since VidCom_example only has 1 video
    'used_videos' : vidlist,
    'trainset' : trainlist,
    'testset' : testlist,
}
VidCom = Dataset(datadict)

Get the ground truth human data

In [ ]:
gt_amp_dva = VidCom.gt_foveation_df["sac_amp_dva"].dropna().values
gt_dur_ms = VidCom.gt_foveation_df["duration_ms"].dropna().values

gt_amp_dva_train = VidCom.train_foveation_df["sac_amp_dva"].dropna().values
gt_dur_ms_train = VidCom.train_foveation_df["duration_ms"].dropna().values

gt_amp_dva_test = VidCom.test_foveation_df["sac_amp_dva"].dropna().values
gt_dur_ms_test = VidCom.test_foveation_df["duration_ms"].dropna().values

Fit distributions to the histograms and scale them appropriately

In [ ]:
ampfit = stats.expon.fit(gt_amp_dva)
ampX = np.linspace(0,50, 200)
ampPDF = stats.expon.pdf(ampX, *ampfit)

durfit = stats.norm.fit(np.log10(gt_dur_ms))
durX = np.linspace(1,4, 200)
durPDF = stats.norm.pdf(durX, *durfit)
print(durfit, ampfit)

c_count, b_count = np.histogram(np.log10(gt_dur_ms), density=False,bins=40)
c_dense, b_dense = np.histogram(np.log10(gt_dur_ms), density=True,bins=40)
FD_fac = c_count.max()/c_dense.max()
c_count, b_count = np.histogram(gt_amp_dva, density=False,bins=60)
c_dense, b_dense = np.histogram(gt_amp_dva, density=True,bins=60)
SA_fac = c_count.max()/c_dense.max()

In [ ]:
print("report with basis e: ", stats.norm.fit(np.log(gt_dur_ms)))
print("expected value is ", np.exp(5.734790631554358 + 0.5 * 0.8380120606761086**2))

## Fig 4
### a & b
Ground truth scanpath statistics.

In [ ]:
# use different color palatte from the functional evaluation (see below)
cl = sns.color_palette("Dark2")

fig, axs = plt.subplots(1,2,dpi=150, figsize=(8,3), sharey=True)
sns.histplot(data=np.log10(gt_dur_ms), kde=False, ax=axs[0], bins=40, color=cl[0])
axs[0].plot(durX, durPDF*FD_fac, color='k', ls=':')
axs[0].set_xticks([1,2,3,4])
axs[0].set_xticklabels([10,100,1000,10000])
axs[0].set_xlabel('Foveation duration [ms]', size=14)
axs[0].set_ylabel('Count', size=14)
sns.histplot(data=gt_amp_dva, kde=False, ax=axs[1], bins=60, color=cl[0])
axs[1].set_xlabel('Saccade amplitude [dva]', size=14)
axs[1].set_xlim([0, 50])
axs[1].plot(ampX, ampPDF*SA_fac, color='k', ls=':', label='Fit')
axs[0].tick_params(labelsize=12)
axs[1].tick_params(labelsize=12)
sns.despine()
plt.tight_layout(); plt.show()

### c & d
Compare the CDF summary statistics of the human data with the models. Show training in transparent and test in opaque.

In [ ]:
cl = sns.color_palette("Dark2")

names = [ 'L.ll', 'L.hl', 'O.ll', 'O.cb']
gtname = 'Human data'
run_ids = [
    'loc_train_molin_64-32-50_2023-03-09-01H-04M-33S_22332349',
    'loc_train_TASEDnet_64-32-50_2023-03-09-01H-02M-20S_22332348',
    'obj_train_molin_64-32-50_2023-03-09-01H-04M-58S_22332350',
    'obj_train_None_64-32-50_2023-03-09-01H-05M-43S_22332351',
]
# prepare custom legend
handles1 = []; handles2 = []
for ind in [0,1,2,3,5]:
    rgb = to_rgb(cl[ind])
    handles1.append(plt.Rectangle((0, 0), 0, 0, facecolor=rgb, edgecolor=rgb))
    rgb = 0.5 + 0.5 * np.array(rgb)  # make whiter
    handles2.append(plt.Rectangle((0, 0), 0, 0, facecolor=rgb, edgecolor=rgb))

fig, axs = plt.subplots(1,2,dpi=150, figsize=(8,3), sharey=True)
for modus in ["train", "test"]:
    c_idx = 0
    print(modus)

    if modus == "train":
        videoset = VidCom.trainset
        gt_amp_dva = VidCom.train_foveation_df["sac_amp_dva"].dropna().values
        gt_dur_ms = VidCom.train_foveation_df["duration_ms"].dropna().values
        kwargs = {"lw":3, "alpha":0.3} #"ls":"dotted", 
    else:
        videoset = VidCom.testset
        gt_amp_dva = VidCom.test_foveation_df["sac_amp_dva"].dropna().values
        gt_dur_ms = VidCom.test_foveation_df["duration_ms"].dropna().values
        kwargs = {"lw":2}#, "ls":"dotted", "alpha":0.5}

    df_res_pxMolin_top0 = pd.read_csv(f'./ScanDy_results/{run_ids[0]}/{modus}res_df_top0.csv')
    # df_res_pxMolin_top0 = df_res_pxMolin_top0[df_res_pxMolin_top0['video'].isin(videoset)]
    px_llf_dur_ms = df_res_pxMolin_top0["duration_ms"].dropna().values
    px_llf_amp_dva = df_res_pxMolin_top0["sac_amp_dva"].dropna().values

    df_res_pxTased_top0 = pd.read_csv(f'./ScanDy_results/{run_ids[1]}/{modus}res_df_top0.csv')
    # df_res_pxTased_top0 = df_res_pxTased_top0[df_res_pxTased_top0['video'].isin(videoset)]
    px_hlf_dur_ms = df_res_pxTased_top0["duration_ms"].dropna().values 
    px_hlf_amp_dva = df_res_pxTased_top0["sac_amp_dva"].dropna().values 

    df_res_objMolin_top0 = pd.read_csv(f'./ScanDy_results/{run_ids[2]}/{modus}res_df_top0.csv')
    # df_res_objMolin_top0 = df_res_objMolin_top0[df_res_objMolin_top0['video'].isin(videoset)]
    obj_llf_dur_ms = df_res_objMolin_top0["duration_ms"].dropna().values
    obj_llf_amp_dva = df_res_objMolin_top0["sac_amp_dva"].dropna().values

    df_res_objNone_top0 = pd.read_csv(f'./ScanDy_results/{run_ids[3]}/{modus}res_df_top0.csv')
    # df_res_objNone_top0 = df_res_objNone_top0[df_res_objNone_top0['video'].isin(videoset)]
    obj_cb_dur_ms = df_res_objNone_top0["duration_ms"].dropna().values
    obj_cb_amp_dva = df_res_objNone_top0["sac_amp_dva"].dropna().values

    nbins = 60
    axs[0].hist(np.log10(px_llf_dur_ms), nbins, density=True, histtype='step', cumulative=True, label=names[0], color=cl[1], **kwargs)
    axs[0].hist(np.log10(px_hlf_dur_ms), nbins, density=True, histtype='step', cumulative=True, label=names[1], color=cl[2], **kwargs)
    axs[0].hist(np.log10(obj_llf_dur_ms), nbins, density=True, histtype='step', cumulative=True, label=names[2], color=cl[3], **kwargs)
    axs[0].hist(np.log10(obj_cb_dur_ms), nbins, density=True, histtype='step', cumulative=True, label=names[3], color=cl[5], **kwargs)
    axs[0].hist(np.log10(gt_dur_ms), nbins, density=True, histtype='step', cumulative=True, label=gtname, color=cl[0], **kwargs)
    axs[0].set_xticks([1,2,3,4])
    axs[0].set_xticklabels([10,100,1000,10000], size=14)
    axs[0].tick_params(labelsize=12)
    axs[0].set_xlabel('Foveation duration [ms]', size=14)
    axs[0].set_ylabel('CDF', size=14)
    axs[1].hist(px_llf_amp_dva, nbins, density=True, histtype='step', cumulative=True, label=names[0], color=cl[1], **kwargs)
    axs[1].hist(px_hlf_amp_dva, nbins, density=True, histtype='step', cumulative=True, label=names[1], color=cl[2], **kwargs)
    axs[1].hist(obj_llf_amp_dva, nbins, density=True, histtype='step', cumulative=True, label=names[2], color=cl[3], **kwargs)
    axs[1].hist(obj_cb_amp_dva, nbins, density=True, histtype='step', cumulative=True, label=names[3], color=cl[5], **kwargs)
    axs[1].hist(gt_amp_dva, nbins, density=True, histtype='step', cumulative=True, label=gtname, color=cl[0], **kwargs)
    axs[1].set_xlabel('Saccade amplitude [dva]', size=14)
    axs[1].tick_params(labelsize=12)
    axs[1].set_xlim([0, 50])
    #axs[1].legend(loc='lower right')
    uf.fix_hist_step_vertical_line_at_end(axs[0])
    uf.fix_hist_step_vertical_line_at_end(axs[1])
axs[1].legend(bbox_to_anchor=(1,0), loc='lower right', 
            frameon=False, handles=[tuple(handles1), tuple(handles2)], labels=["Test set", "Training set"],
            title="", handlelength=3, handler_map={tuple: HandlerTuple(ndivide=None, pad=0)}, fontsize=11)

sns.despine(); plt.tight_layout(); plt.show()


In [ ]:
np.max(px_hlf_dur_ms)

In [ ]:
# create legend separately
fig, ax = plt.subplots(1,1,dpi=150, figsize=(4,3))
ax.plot([0,1], [0,0], lw=2, label=gtname, color=cl[0])
ax.plot([0,1], [0,0], lw=2, label="L.ll model", color=cl[1])#names[0])
ax.plot([0,1], [0,0], lw=2, label="L.hl model", color=cl[2])#names[1])
ax.plot([0,1], [0,0], lw=2, label="O.ll model", color=cl[3])#names[2])
ax.plot([0,1], [9,9], lw=2, label="O.cb model", color=cl[5])#names[3])
plt.legend()
plt.show()

## Fig 5

### a) Foveation categories over time for human data

In [ ]:
cl = sns.color_palette()

BDIR_per_frames_train = np.zeros((4,300))
BDIR_per_frames = np.zeros((4,300))
bdir_to_row = {'B': 0, 'D': 1, 'I': 2, 'R': 3}
for index, row in VidCom.train_foveation_df.iterrows():
    BDIR_per_frames_train[bdir_to_row[row['fov_category']], row['frame_start']:row['frame_end']+1] += 1
for index, row in VidCom.test_foveation_df.iterrows():
    BDIR_per_frames[bdir_to_row[row['fov_category']], row['frame_start']:row['frame_end']+1] += 1

combined_per_frames_train = np.sum(BDIR_per_frames_train, axis=0)
combined_per_frames = np.sum(BDIR_per_frames, axis=0)

fig, ax = plt.subplots(dpi=200, figsize=(4,2.5))
ax.plot(BDIR_per_frames_train[0] / combined_per_frames_train * 100, color=cl[0], lw=2, ls='-', alpha=0.25)
ax.plot(BDIR_per_frames_train[1] / combined_per_frames_train * 100, color=cl[1], lw=2, ls='-', alpha=0.25)
ax.plot(BDIR_per_frames_train[2] / combined_per_frames_train * 100, color=cl[2], lw=2, ls='-', alpha=0.25)
ax.plot(BDIR_per_frames_train[3] / combined_per_frames_train * 100, color=cl[3], lw=2, ls='-', alpha=0.25)
ax.plot(BDIR_per_frames[0] / combined_per_frames * 100, color=cl[0], label='Background')
ax.plot(BDIR_per_frames[1] / combined_per_frames * 100, color=cl[1], label='Detection')
ax.plot(BDIR_per_frames[2] / combined_per_frames * 100, color=cl[2], label='Inspection')
ax.plot(BDIR_per_frames[3] / combined_per_frames * 100, color=cl[3], label='Revisit')
ax.set(xlabel='Time [frames]', ylabel='Percentage') # title='Ground truth foveation category over time', 
# plt.legend(); 
sns.despine(); plt.show()


For the supplement, check the temporally resolved ratios for all models! 

In [ ]:
for model_idx in range(4):
    runid = run_ids[model_idx]
    name = names[model_idx]
    
    if 'obj_' in runid:
        model = ObjectModel(VidCom)
        parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_inobj']
    elif 'loc_' in runid:
        model = LocationModel(VidCom)
        parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_dva']
    else:
        raise NotImplementedError("Only Object- and Location-based are implemented")

    train_df = pd.read_csv(f'ScanDy_results/{runid}/trainres_df_top{0}.csv')
    test_df = pd.read_csv(f'ScanDy_results/{runid}/testres_df_top{0}.csv')
    model.result_df = pd.concat([train_df, test_df], ignore_index=True)

    ratios = model.functional_event_courses(videos_to_eval="test")
    ratios_train = model.functional_event_courses(videos_to_eval="train")

    print(name)
    fig, ax = plt.subplots(dpi=200, figsize=(4,2.5))
    ax.plot(ratios_train[0], color=cl[0], lw=2, ls='-', alpha=0.25)
    ax.plot(ratios_train[1], color=cl[1], lw=2, ls='-', alpha=0.25)
    ax.plot(ratios_train[2], color=cl[2], lw=2, ls='-', alpha=0.25)
    ax.plot(ratios_train[3], color=cl[3], lw=2, ls='-', alpha=0.25)
    ax.plot(ratios[0], color=cl[0], label='Background')
    ax.plot(ratios[1], color=cl[1], label='Detection')
    ax.plot(ratios[2], color=cl[2], label='Inspection')
    ax.plot(ratios[3], color=cl[3], label='Revisit')
    # ax.set(title=name, xlabel='Time [frames]', ylabel='Percentage', ylim=[0,100]) # title='Ground truth foveation category over time', 
    ax.set(xlabel='Time [frames]', ylabel='Percentage', ylim=[0,100]) # title='Ground truth foveation category over time', 
    sns.despine(); plt.show()

In main paper, reduce this to the barplot of the mean ratios, which is calculated afterwards.
First, look into the train/test boxplots!

## Fig 6: BDIR fractions relative to the human data


In [ ]:
print("Fraction of the stimulus time taken up by foveation events in the human data: ", VidCom.get_foveation_ratio())
print("This is 100% for the models.")

In [ ]:
fig, axes = plt.subplots(1,4, sharex=True, sharey=True, figsize=(8,3), dpi=200)
report_par_decimals = [3,3,2,1,2]

subjects_fovcat_train = list(VidCom.get_fovcat_ratio("train").values())
subjects_fovcat_test = list(VidCom.get_fovcat_ratio("test").values())

mean_fovcats_train = []
mean_fovcats_test = []

evol = Evolution(lambda x: x, ParameterSpace(['mock'], [[0, 1]]))
for ax_i, ax in enumerate(axes.flat):
    runid = run_ids[ax_i]
    print(runid)
    DILLNAME = f'{runid}.dill'
    if 'obj_' in runid:
        model = ObjectModel(VidCom)
        parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_inobj']
    elif 'loc_' in runid:
        model = LocationModel(VidCom)
        parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_dva']
    evol = evol.loadEvolution(f'ScanDy_results/{runid}/{DILLNAME}')
    df_evol = evol.dfEvolution(outputs=True).copy()
    df_top32 = df_evol.sort_values('score', ascending=False)[:32]
    # read in test simulation results
    test_fovcats = []
    for i in range(32):
        model.result_df = pd.read_csv(f'ScanDy_results/{runid}/testres_df_top{i}.csv')
        test_fovcats.append(np.array(list(model.get_fovcat_ratio().values())) )
    df_top32['test_fov_cat'] = test_fovcats

    print(runid, "mean params")
    for i, par in enumerate(parameters):
        top_par = round(df_top32[par].iloc[0],report_par_decimals[i])
        print(f'{par} = ${round(df_top32[par].mean(),report_par_decimals[i])} \pm {round(df_top32[par].std(),report_par_decimals[i])}$     & {top_par}' )
    print("Balancing of fitness function")
    print("$d_{FD} =", f'{round(df_top32["f0"].mean(), 3)} \pm {round(df_top32["f0"].std(), 3)}$   top0: {round(df_top32["f0"].iloc[0], 3)}' )
    print("$d_{SA} =", f'{round(df_top32["f1"].mean(), 3)} \pm {round(df_top32["f1"].std(), 3)}$   top0: {round(df_top32["f1"].iloc[0], 3)}' )


    fovcats_train = [df_top32['fov_cat'].iloc[indv] for indv in range(32)]
    mean_fovcats_train.append(np.mean(np.array(fovcats_train), axis=0))
    print("Train, ", names[ax_i], np.mean(np.array(fovcats_train), axis=0))
    train_rel_fovcat = [df_top32['fov_cat'].iloc[indv] / np.array(subjects_fovcat_train) for indv in range(32)]
    
    fovcats_test = [df_top32['test_fov_cat'].iloc[indv] for indv in range(32)]
    mean_fovcats_test.append(np.mean(np.array(fovcats_test), axis=0))
    print("Test, ", names[ax_i], np.mean(np.array(fovcats_test), axis=0))
    print("Test ratio to GT, ", names[ax_i], np.mean(np.array(fovcats_test), axis=0) / np.array(subjects_fovcat_test))
    test_rel_fovcat = [df_top32['test_fov_cat'].iloc[indv] / np.array(subjects_fovcat_test) for indv in range(32)]
    df_train_fovcats_rel_gt = pd.DataFrame(index=[f'Indv. Top {t}' for t in range(32)] , 
                                     columns=['B', 'D', 'I', 'R'], data=train_rel_fovcat
                                     ).assign(Data='training set')
    df_test_fovcats_rel_gt = pd.DataFrame(index=[f'Indv. Top {t}' for t in range(32)] , 
                                          columns=['B', 'D', 'I', 'R'], data=test_rel_fovcat
                                          ).assign(Data='test set')
    cdf = pd.concat([df_test_fovcats_rel_gt, df_train_fovcats_rel_gt])
    mdf = pd.melt(cdf, id_vars=['Data'], var_name=['Foveation category'])

    sns.violinplot(x="Foveation category", y="value", hue="Data", data=mdf, ax=ax, split=True, linewidth=1, palette={"test set": cl[0], "training set": ".85"})    
    # df_fovcats_rel_gt.boxplot(ax=ax, grid=False)
    handles = []
    for ind, violin in enumerate(ax.findobj(PolyCollection)):
        rgb = to_rgb(cl[ind // 2])
        if ind % 2 != 0:
            rgb = 0.5 + 0.5 * np.array(rgb)  # make whiter
        violin.set_facecolor(rgb)
        handles.append(plt.Rectangle((0, 0), 0, 0, facecolor=rgb, edgecolor=rgb))
    ax.set_yscale('log')
    ax.set_yticks([0.25,0.5,1,1.5,2])
    ax.set_yticklabels([0.25,0.5,1,1.5,2])
    ax.axhline(1, color='k', ls=':')

    if (ax_i%4)==0:
        ax.set_ylabel('Fraction of viewing time\nrelative to human data', size=12)
        # ax.set_xlabel('Fov. category', size=14)
    else:
        ax.set_ylabel('')
    if ax_i==3:
        ax.legend(bbox_to_anchor=(1,0), loc='lower right', 
                  frameon=False, handles=[tuple(handles[::2]), tuple(handles[1::2])], labels=["Test set", "Training set"],
                  title="", handlelength=3, handler_map={tuple: HandlerTuple(ndivide=None, pad=0)}, fontsize=9)
    else:
        ax.get_legend().remove()
    ax.set_xlabel('')
    ax.set_title(names[ax_i])
fig.text(0.55, 0.0, 'Foveation category', size=12, ha='center')    
plt.tight_layout()
sns.despine(); plt.show()

In [ ]:
subjects_fovcat_train

In [ ]:
subjects_fovcat_test

## Fig 5b Barplots
Use the means from the distributions plotted in Fig 6

In [ ]:
# names_r = names[::-1]
names_r = ["O.cb model", "O.ll model", "L.hl model", "L.ll model"]
mean_fovcats_train_r = mean_fovcats_train[::-1]
mean_fovcats_test_r = mean_fovcats_test[::-1]

subjects_fovcat_train = list(VidCom.get_fovcat_ratio("train").values())
subjects_fovcat_train = list(VidCom.get_fovcat_ratio("train").values())

df_fovcats_train = pd.DataFrame(index=names_r + ['Human\ndata'], 
                        columns=['B', 'D*', 'I', 'R'],
                        data= mean_fovcats_train_r + [subjects_fovcat_train/sum(subjects_fovcat_train)])
df_fovcats_test = pd.DataFrame(index=names_r + ['Human\ndata'], 
                        columns=['B', 'D*', 'I', 'R'],
                        data= mean_fovcats_test_r + [subjects_fovcat_test/sum(subjects_fovcat_test)])

# fig, ax = plt.subplots(figsize=(3.5,2.5),dpi=150)
fig, ax = plt.subplots(figsize=(3.7,2.5),dpi=150)
df_fovcats_train.plot.barh(stacked=True, ax=ax,rot=0, legend=False)#.legend(bbox_to_anchor=(0.3, 1.5))
ax.set_frame_on(False); ax.set(xlabel = 'Ratio of events')#, title = 'Trainset')
print("Trainset")
plt.tight_layout(); plt.show()
fig, ax = plt.subplots(figsize=(3.7,2.5),dpi=150)
df_fovcats_test.plot.barh(stacked=True, ax=ax,rot=0, legend=False)#.legend(bbox_to_anchor=(0.3, 1.5))
ax.set_frame_on(False); ax.set(xlabel = 'Ratio of events')#, title = 'Testset')
print("Testset")
plt.tight_layout(); plt.show()

# Supplemental material

In [ ]:
videos = sorted(VidCom.gt_foveation_df.video.unique())

# go through all videos and print a list that says for each if it is in the training or test set 
print(f"Name\t \t#Subjects\t \tTrain/test set")
for video in videos:
    nsubj = len(VidCom.gt_foveation_df[VidCom.gt_foveation_df.video==video].subject.unique())
    if video in VidCom.trainset:
        print(f"{video}\t \t{nsubj}\t \ttrain")
    else:
        print(f"{video}\t \t{nsubj}\t \ttest")


## S1 & S2 are about parameter understanding and robustness for both models

In [ ]:
# where to get parameters from? --> previous evolution!
runid = 'loc_train_molin_64-32-50_2023-03-09-01H-04M-33S_22332349'
model = LocationModel(VidCom)
parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_dva']
par_sym = [r'DDM thres. $\theta$', r'DDM noise $s$', r'Sensitivity spread $\sigma_S$', r'IOR slope $1/r$', r'IOR spread $\sigma_I$']
relative_par_vals = [0.5, 0.75, 0.9, 1.1, 1.25, 2]

uf.plot_var_pars(model, 'ScanDy_results/', runid, parameters, par_sym, relative_par_vals)

### Same for object based model...

In [ ]:
# where to get parameters from? --> previous evolution!
runid = 'obj_train_molin_64-32-50_2023-03-09-01H-04M-58S_22332350'
model = ObjectModel(VidCom)
parameters = ['ddm_thres', 'ddm_sig', 'att_dva', 'ior_decay', 'ior_inobj']
par_sym = [r'DDM thres. $\theta$', r'DDM noise $s$', r'Sensitivity spread $\sigma_S$', r'IOR slope $1/r$', r'Object IOR $\xi$']
relative_par_vals = [0.5, 0.75, 0.9, 1.1, 1.25, 2]

uf.plot_var_pars(model, runid, parameters, par_sym, relative_par_vals)

## S3-6: Parameter spaces of the 4 models

In [ ]:
run_ids = [
    'loc_train_molin_64-32-50_2023-03-09-01H-04M-33S_22332349',
    'loc_train_TASEDnet_64-32-50_2023-03-09-01H-02M-20S_22332348',
    'obj_train_molin_64-32-50_2023-03-09-01H-04M-58S_22332350',
    'obj_train_None_64-32-50_2023-03-09-01H-05M-43S_22332351',
]
for runid in run_ids:
    DILLNAME = f'{runid}.dill'
    evol = Evolution(lambda x: x, ParameterSpace(['mock'], [[0, 1]]))
    evol = evol.loadEvolution(f'ScanDy_results/{runid}/{DILLNAME}')
    evol.info()


### S7 see above